1. Diferenças entre AdaBoost e GBM:
AdaBoost (Adaptive Boosting) e GBM (Gradient Boosting Machines) são ambos métodos de boosting, mas possuem diferenças importantes:

Técnica de Boosting:

AdaBoost ajusta pesos dos exemplos incorretamente classificados, dando mais atenção a esses erros em cada iteração.
GBM minimiza uma função de perda em cada iteração, ajustando novos modelos para reduzir o erro residual.
Tipos de Erros:

AdaBoost trabalha diretamente ajustando pesos em função da taxa de erro.
GBM utiliza o gradiente da função de perda para ajustar os modelos, o que é mais flexível.
Taxa de Aprendizado:

AdaBoost não tem uma taxa de aprendizado explícita. Cada iteração contribui igualmente ao modelo final.
GBM usa uma taxa de aprendizado explícita para controlar o impacto de cada modelo ajustado, oferecendo controle mais fino.
Robustez a Outliers:

AdaBoost pode ser sensível a outliers, pois pode continuar focando em exemplos difíceis, mesmo se forem ruído.
GBM tem maior controle sobre o ajuste a outliers com a função de perda escolhida e hiperparâmetros.
Desempenho Computacional:

AdaBoost é geralmente mais rápido para ajustar por não otimizar uma função de perda complexa.
GBM pode ser mais lento devido à otimização iterativa da função de perda e cálculo de gradientes.

In [22]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

from scipy.stats import ks_2samp
from sklearn.metrics import accuracy_score, roc_auc_score

import pandas as pd
import numpy as np

from sklearn.metrics  import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

In [6]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

In [8]:
#criando a df de test para mais adiante usar nas métricas
df_test = pd.DataFrame(X_test)
df_test['y'] = y_test
df_test

,0,1,2,3,4,5,6,7,8,9,y
0,0.330046,-0.000480,0.818116,0.428214,-2.503947,0.120481,0.807893,0.602121,-0.865190,-0.153320,-1.0
1,-0.240491,-0.060763,0.532866,0.764401,-0.856083,-1.682340,0.653069,0.930255,0.405183,0.204002,-1.0
2,-0.822766,-1.276874,-0.226437,-1.016053,-0.657295,0.293991,0.620053,0.779634,0.454984,-1.303436,-1.0
3,-0.564450,-0.697132,1.037840,1.560154,-1.460206,-1.939676,0.526586,1.660339,-0.363809,-0.031045,1.0
4,-0.363622,1.241802,-1.668672,0.613210,0.043469,0.116548,-0.578605,-1.264679,-0.117487,-0.485187,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.070208,0.328849,1.006085,0.440949,-0.692274,0.237091,-0.565192,1.893955,1.277944,0.789953,-1.0
9996,-1.692417,0.968971,1.371423,-0.144225,0.503164,-0.228220,0.153762,1.872044,2.153554,0.006797,1.0
9997,0.199865,0.261346,-0.127987,-0.298503,-0.364240,-1.370135,0.769663,-0.517183,0.079694,1.016127,-1.0
9998,-0.151673,-1.425200,1.070922,0.938801,1.373176,-0.583906,-0.292270,-1.206766,-1.047465,0.007588,-1.0


In [10]:
#criando a df de treino para mais adiante usar nas métricas
df_train = pd.DataFrame(X_train)
df_train['y'] = y_train
df_train

,0,1,2,3,4,5,6,7,8,9,y
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599,1.0
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096,-1.0
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359,1.0
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303,-1.0
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1995,0.581415,-0.241553,-0.294910,-0.138690,0.909687,-0.229796,-0.512237,0.625726,-0.592289,-0.736915,-1.0
1996,-0.061139,-0.497147,0.432102,0.977877,0.120829,0.249489,0.067397,-0.176741,0.987143,0.281150,-1.0
1997,1.902813,-0.242745,-0.911743,-1.327828,0.023258,-1.663408,0.569723,-0.788811,0.205704,0.342916,1.0
1998,-0.755901,0.919071,1.534184,1.063159,-0.708948,1.854091,0.682429,-0.877648,-0.980262,2.394037,1.0


In [29]:
X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

est = GradientBoostingRegressor(n_estimators=100, 
                                learning_rate=0.1, 
                                max_depth=1, 
                                random_state=0, 
                                loss='squared_error').fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

5.009154859960321

3. Cinco Hiperparâmetros Importantes no GBM:
n_estimators: O número de árvores no modelo. Aumentar esse valor melhora o ajuste, mas também aumenta o risco de overfitting.
learning_rate: A taxa de aprendizado. Controla o impacto de cada árvore adicionada ao modelo. Valores baixos exigem mais árvores.
max_depth: A profundidade máxima de cada árvore. Controla a complexidade do modelo.
subsample: A fração dos dados usada para treinar cada árvore. Um valor menor que 1 pode introduzir aleatoriedade e ajudar a evitar overfitting.
min_samples_split: O número mínimo de amostras necessárias para dividir um nó, ajudando a controlar o tamanho das árvores.

In [33]:
%%time

estimators     = [10, 100, 1000, 10000]
learning_rates = [0.01, 0.03, 0.06, 0.1]
max_depths     = [1, 3, 6, 9]

grid_search = []

for n in estimators:
    for rate in learning_rates:
        for depth in max_depths:
            est = GradientBoostingRegressor(n_estimators=n, 
                                            learning_rate=rate, 
                                            max_depth=depth, 
                                            random_state=0, 
                                            loss='squared_error').fit(X_train, y_train)
            grid_search.append([n, rate, depth, mean_squared_error(y_test, est.predict(X_test))])
            
(pd.DataFrame(data=grid_search, 
              columns=['n_estimators', 'learning_rate', 'max_depth', 'mean_squared_error'])
   .sort_values(by='mean_squared_error', 
                ascending=True, 
                ignore_index=True))

CPU times: total: 29.3 s
Wall time: 1min 22s


,n_estimators,learning_rate,max_depth,mean_squared_error
0,1000,0.03,1,3.625209
1,1000,0.06,1,3.667692
2,10000,0.10,3,3.749941
3,1000,0.10,3,3.750008
4,100,0.10,3,3.790073
...,...,...,...,...
59,10,0.03,1,21.909960
60,10,0.01,9,22.402269
61,10,0.01,6,22.472923
62,10,0.01,3,22.902975


5. Diferença entre GBM e Stochastic GBM:
A principal diferença entre o GBM tradicional e o Stochastic GBM (SGBM) é o uso de amostragem aleatória:

GBM usa todo o conjunto de dados em cada iteração para ajustar o modelo.
Stochastic GBM aplica amostragem aleatória de uma fração dos dados (definida pelo hiperparâmetro subsample) em cada